In [4]:
import pandas as pd
from darknet import Darknet
from utils import Image, do_detect, load_class_names, plot_boxes
import cv2
import numpy as np

ModuleNotFoundError: No module named 'torch'

In [2]:
cfgfile = 'cfg/yolo.cfg'
weightfile = 'yolo.weights'
imgfile = 'traffic_light.png'
width = 1920
height = 1080

In [3]:
m = Darknet(cfgfile)
m.load_weights(weightfile)
# m.print_network()

/Users/anton/Documents/GitHub/vision_hack/cfg.py:175: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  conv_model.weight.data.copy_(torch.from_numpy(buf[start:start+num_w])); start = start + num_w
/Users/anton/Documents/GitHub/vision_hack/cfg.py:157: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  conv_model.weight.data.copy_(torch.from_numpy(buf[start:start+num_w])); start = start + num_w


In [4]:
if m.num_classes == 20:
    namesfile = 'data/voc.names'
elif m.num_classes == 80:
    namesfile = 'data/coco.names'
else:
    namesfile = 'data/names'

In [5]:
class_names = load_class_names(namesfile)
traffic_light_index = class_names.index('traffic light')

In [23]:
def detect(frame):
    img = Image.fromarray(frame).convert('RGB')
    sized = img.resize((m.width, m.height))
    boxes = do_detect(m, sized, 0.05, 0.4, 0)
    traffic_light_boxes = [box for box in boxes if box[6] == traffic_light_index]
    boxes_absolute = [{'x1' : tlb[0] * width,
                      'y1' : tlb[1] * height,
                      'x2' : x1 + tlb[2] * width,
                      'y2' : y1 + tlb[3] * height} 
                     for tlb in traffic_light_boxes]
    df = pd.DataFrame(boxes_absolute)
    return plot_boxes(img, traffic_light_boxes, class_names=class_names), df

In [25]:
file_path = 'akn.036.020.left.avi'
file_out = './akn.036.020.left.segmented.avi'
file_out_csv = './akn.036.020.left.csv'

video = cv2.VideoCapture(file_path)
fps = int(video.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'H264')

out_video = cv2.VideoWriter(file_out, fourcc, fps=fps, frameSize=(1920, 1080))

if not (video.isOpened() and video.get(cv2.CAP_PROP_FRAME_COUNT) > 1):
    print('no video')

# length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
# print(length)
# video.set(cv2.CAP_PROP_POS_FRAMES, 0)

frames_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
current_frame = 0
boxes_dataframes = list()
while video.isOpened() and current_frame < frames_count:
    ret, frame = video.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plot, boxes_df = detect(frame)
        boxes_df['frame'] = current_frame
        boxes_dataframes.append(boxes)
        pred_frame = np.array(plot)
        pred_frame = cv2.cvtColor(pred_frame, cv2.COLOR_RGB2BGR)
        out_video.write(pred_frame)
        if current_frame % 24 == 0:
            print('Handled frame:', current_frame)
    current_frame += 1
    # else:
    #     print('error')
pd.concat(boxes_dataframes).to_csv(file_out_csv)
print('Releasing handles')
video.release()
out_video.release()

Handled frame: 24
Handled frame: 48
Handled frame: 72
Handled frame: 96
Handled frame: 120
Handled frame: 144
Handled frame: 168
Handled frame: 192
Handled frame: 216
Handled frame: 240
Handled frame: 264
Handled frame: 288
Releasing handles


In [16]:
print(dir(out_video))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'get', 'isOpened', 'open', 'release', 'set', 'write']


In [18]:
out_video.get()

TypeError: Required argument 'propId' (pos 1) not found

In [1]:
traffic_light_index

NameError: name 'traffic_light_index' is not defined